
## Introduction

This tutorial was originally prepared for the [SEA Scientific Software Conference and Tutorials 2019](https://sea.ucar.edu/conference/2019). The abstract of the workshop can be accessed from [here](https://sea.ucar.edu/event/parallel-analog-ensemble-forecasts-ensemble-toolkit-hpc).

This hands-on exercise will help you generate your first Analog Ensemble (AnEn) and guide you through your first use of the `RAnEn` library. You will learn how to load and format your data, generate a configuration file, and generate an ensemble of analogs. You will learn how to use these functions:

- `generateConfiguration`
- `generateAnalogs`

## R Set Up

First, let's load the R libraries we will use.



In [ ]:
library(RAnEn)
library(maps)

# Check your version
stopifnot(packageVersion('RAnEn') >= "3.3.1")


## Load Data

Next, let's load our data. Our data comes from METAR sites distributed across the continental U.S. The dataset is graciously provided by NCAR for our use today.



In [ ]:
file.name <- 'data-METAR-CONUS.Rdata'

if (!file.exists(file.name)) {
  cat('Downloading from the data server which might be slow ...\n')
  download.file(url = paste('https://prosecco.geog.psu.edu/', file.name, sep = ''),
                destfile = file.name)
}

load(file.name)
rm(file.name)


## Define Parameters

Next, let's define our parameters.



In [ ]:
test.start <- 400
test.end <- 457
search.start <- 1
search.end <- 399
selected.stations <- 1:10


## How to Generate a Configuration File

In this section, we will learn how to generate a configuration file. The configuration file defines all the parameters needed to generate your ensemble of analogs and will be passed to the 'generateAnalogs' function. We will walk through the meaning of each of the parameters.



In [ ]:
# For this experiment, we set the configuration mode to
# 'Independent Search' which indicates we are searching
# for analogs at the same point in space, over a small
# time window. This is the original implementation utilized
# by DelleMonache et al. (2013).
#
config <- generateConfiguration('independentSearch')

# Define the forecasts, forecast lead times (FLT), and time of the forecast
config$forecasts <- forecasts$Data[,selected.stations , , , drop = F]
config$forecast_times <- forecasts$Times
config$flts <- forecasts$FLTs

# Set preserve similarity to TRUE so we can generate some figures that aid analysis
config$preserve_similarity <- T

# Define our observations and the valid time of those observations
config$search_observations <- observations$Data[, selected.stations, , drop = F]
config$observation_times <- observations$Times

# Define the test and search times
config$test_times_compare <- config$forecast_times[test.start:test.end]
config$search_times_compare <- config$forecast_times[search.start:search.end]

# Number of ensemble members and similarity records we want to keep
config$num_members <- 11

# How much information you want printed to the screen in the
# process of generating analogs. 1 is the least and 5 is the most.
# See help for additional specifics.
#
config$verbose <- 3

# Set circular variable if there is any
if ('ParameterCirculars' %in% names(forecasts)) {
  config$circulars <- unlist(lapply(forecasts$ParameterCirculars, function(x) {
    return(which(x == forecasts$ParameterNames))}))
}


## Generate an Analog Ensemble
Next, let's generate our ensemble of analogs!



In [ ]:
AnEn <- generateAnalogs(config)


Remember what we said about the parameter `config$verbose`. You control the level of description coming out of the AnEn generation process. Increase the verbose setting to receive more information printed to screen (max verbose value is 5) or decrease the number to see less (minimum verbose value is 1).

Congratulations, you have now generated your first ensemble of analogs!

## What does the AnEn output data structure look like?



In [ ]:
AnEn


The dimensions indicate the number of stations, number of days, forecast lead times, number of ensemble members, and the third variable stores the value of the analog chosen [1], the station index [2] and the time index [3]. More information can be found in the documentation of `RAnEn::generateAnalogs`. This is how you view the values of your analog ensemble members:



In [ ]:
AnEn$analogs[1,1,1,,1]


## Visualize Our Results

The `RAnEn` package comes with a number of visualization tools. Let's use some of them and discuss the plots they create.

### Plot a Sample Day

First, let's take a look at a sample prediction from the AnEn using `RAnEn::plotAnalogSample`.



In [ ]:
fcst.id <- which(forecasts$ParameterNames == observations$ParameterNames[config$observation_id])
plotAnalogSample(AnEn, config, i.station = 1, i.test.day = 1,
                 obs.id = config$observation_id, fcst.id = fcst.id,
                 parameter.names = forecasts$ParameterNames,
                 flts.fraction = 3600, legend.texts = c("Obs","Fcst"))



Let's disuss this figure.

### Understanding Analog Selection

Recall how the AnEn technique works. Let's take a look at the current deterministic prediction, historical repository of deterministic predictions, corresponding observations, and how the AnEn selection is performed in further depth.



In [ ]:
plotAnalogSelection(AnEn, config, i.station = 1, i.test.day = 4, i.flt = 1,
                    parameter.names = forecasts$ParameterNames)


Now that we have seen analog selection, what factors could influence this?

## Optimal Predictor Weighting

Next, let's explore optimal predictor weighting. The `RAnEn` package has a simple way to define the weights that the user would like to implement within the configuration file. Let's add one more parameter to our configuration file that will influence the weight placed on the predictors.



In [ ]:

weights <- c(0.7, 0.1, 0.1, 0.1)

# To reset to equal weights
# weights <- rep(1, dim(config$forecasts)[1])

config$weights <- weights


Generate new analogs using our weights:



In [ ]:
AnEn.weights <- generateAnalogs(config)


### How have the predictions and analog selection changed?



In [ ]:
plotAnalogSample(AnEn.weights, config, i.station = 1, i.test.day = 4,
                 obs.id = config$observation_id, fcst.id = fcst.id,
                 forecasts$ParameterNames, flts.fraction = 3600,
                 legend.texts = c("Obs","Fcst"))

In [ ]:
plotAnalogSelection(AnEn.weights, config, i.station = 1, i.test.day = 4,
                    i.flt = 1, forecasts$ParameterNames)


Your turn! Repeat this section of code on weighting on your own with different weights of your choosing. What do you observe?

## Operational Search

Next, we'll walk through a new feature called 'operational search.'

Up until this point, we have been defining the historical repository, the search space (also called the training dataset by some but note that this language can become confusing to individuals also using machine learning or artificial intelligence techniques) as a set length of time. For example, with our CONUS METAR dataset, we have been using one year of data for searching and three months for testing. Now, using the operational search functionality, we will sequentially add days to the historical search repository. This is more akin to the real world scenario because, in the real world, we **do** have information from yesterday to include and give us insight.

### Generate analogs using the operational search functionality



In [ ]:
config$operational <- TRUE
AnEn.opr <- generateAnalogs(config)


This concludes our exercise covering the `RAnEn` fundamentals including:

- Generating your first ensemble of analogs using the AnEn technique
- Generating the AnEn with the predictor parameters weighted
- Generating the AnEn using the Operational Search functionality


## Going Further

Take an opportunity to explore the behavior of the AnEn technique using this sample dataset. Explore the following questions:

- What do you notice when you increase/decrease the number of ensemble members?
- Vary the size of the historical search repository (also often referred to as the training). What do you observe?
- What happens when you change the predictor weights?
- What happens when you utilzie the operational case option?
